In [ ]:
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Activation, Reshape, Concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
from keras.utils import to_categorical

# Define the generator network
def build_generator(z_dim, password_dim, conditioning_dim):
    model = Sequential()

    # Input layer
    input_layer = Input(shape=(z_dim,))
    x = Dense(128)(input_layer)
    x = LeakyReLU(alpha=0.2)(x)

    # Conditioning layer
    conditioning_layer = Input(shape=(conditioning_dim,))
    conditioning = Dense(128)(conditioning_layer)
    conditioning = LeakyReLU(alpha=0.2)(conditioning)
    x = Concatenate()([x, conditioning])

    # Output layer
    x = Dense(password_dim, activation='softmax')(x)
    output_layer = Reshape((password_dim,))(x)

    model = Model(inputs=[input_layer, conditioning_layer], outputs=output_layer)
    return model

# Define the discriminator network
def build_discriminator(password_dim, conditioning_dim):
    model = Sequential()

    # Input layer
    input_layer = Input(shape=(password_dim,))
    x = Reshape((password_dim, 1))(input_layer)

    # Conditioning layer
    conditioning_layer = Input(shape=(conditioning_dim,))
    conditioning = Dense(password_dim)(conditioning_layer)
    conditioning = Reshape((password_dim, 1))(conditioning)
    x = Concatenate()([x, conditioning])

    # Hidden layers
    x = Dense(128)(x)
    x = LeakyReLU(alpha=0.2)(x)

    # Output layer
    x = Dense(1, activation='sigmoid')(x)
    output_layer = Reshape((1,))(x)

    model = Model(inputs=[input_layer, conditioning_layer], outputs=output_layer)
    return model

# Define the CGAN model
def build_cgan(generator, discriminator):
    discriminator.trainable = False

    input_layer = Input(shape=(z_dim,))
    conditioning_layer = Input(shape=(conditioning_dim,))
    generated_password = generator([input_layer, conditioning_layer])
    validity = discriminator([generated_password, conditioning_layer])

    model = Model(inputs=[input_layer, conditioning_layer], outputs=validity)
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5), metrics=['accuracy'])
    return model

# Preprocess the data
def preprocess_data(data):
    # Convert passwords to one-hot encoded vectors
    password_vectors = []
    for password in data:
        password_vector = [to_categorical(ord(c), num_classes=256) for c in password]
        password_vectors.append(password_vector)
    password_vectors = np.array(password_vectors)

    # Combine user and password features
    user_feature = np.array([ord(c) for c in user])
    user_feature = np.tile(user_feature, (len(data), 1))
    password_feature = np.arange(len(data)).reshape(len(data), 1)
    conditioning = np.concatenate((user_feature, password_feature), axis=1)

    # Split the data into training and testing sets
    training_data_size = int(len(data) * 0.8)
    training_data = password_vectors[:training_data_size]
    training_conditioning = conditioning[:training_data_size]
    testing_data = password_vectors
